## Useful resources

### Documentation

- [OpenAI Gym](https://gym.openai.com/docs/#environments)
- [Retro](https://retro.readthedocs.io/en/latest/python.html)

### Reinforcement implementations

- https://github.com/h3nnn4n/Reverse-Engineering-the-GameBoy-Tetris/blob/master/mem_locations.md
- https://adventuresinmachinelearning.com/reinforcement-learning-tutorial-python-keras/
- https://yanpanlau.github.io/2016/07/10/FlappyBird-Keras.html
- https://github.com/keras-rl/keras-rl

In [ ]:
# Standard
from pathlib import Path
import threading
import time

# Extra
import retro
from IPython.display import display
import ipywidgets as widgets

In [ ]:
TETRIS_PATH = Path(".") / "games"

In [ ]:
retro.data.Integrations.add_custom_path(TETRIS_PATH.resolve())

In [ ]:
info_obj = {
    "lines_cleared": 0,
    #'frame_delay_remaining': 4,
    "level": 0,
    "frame_delay": 34,
    "lines_cleared_d1": 0,
    "preview_piece": 4,
    "screen_state": 0,
    "score": 0,
    "lines_cleared_d3": 25,
    "lines_cleared_d2": 25,
}

In [ ]:
def create_widgets():
    # Create info outputs
    for key in info_obj.keys():
        all_widgets[key] = widgets.IntText(
            description=f"{key}:",
            disabled=True,
            layout=widgets.Layout(width="200px"),
            style={"description_width": "140px"},
        )
    
    # Create stop button
    def on_button_clicked(_):
        global stop_threads
        stop_threads = True

    button = widgets.Button(description='Stop')
    button.on_click(on_button_clicked)

    all_widgets["Button"] = button
        
    w = widgets.GridBox(
        [v for k, v in all_widgets.items()],
        layout=widgets.Layout(grid_template_columns="repeat(2, 300px)"),
    )
    return w

In [ ]:
def update_widgets(info):
    for k, v in info_obj.items():
        all_widgets[k].value = info[k]

In [ ]:
def run_game(id, stop):
    print("Starting Tetris...")
    env = retro.make(game="Tetris-GameBoy", inttype=retro.data.Integrations.ALL)
    obs = env.reset()    
    while True:
        obs, rew, done, info = env.step(env.action_space.sample())
        env.render()
        update_widgets(info)
        if info["screen_state"] != 0:
            break
        if done:
            obs = env.reset()        
        if stop():
            print("Exiting loop.")
            break
    #env.render(close=True)            
    env.close()
    print("Execution Interrupted.")

In [ ]:
all_widgets = {}
stop_threads = False
worker = threading.Thread(target=run_game, args=(id, lambda: stop_threads))
display(create_widgets())

In [ ]:
worker.start()